In [25]:
# 💬 Instalar AutoGluon si es necesario
#%pip install autogluon.timeseries

import pandas as pd
import numpy as np
from tqdm import tqdm
import gc, os, shutil
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame

In [26]:
def percentage_safe(numerator: pd.Series, denominator: pd.Series, dtype='float32', fillna=None) -> pd.Series:
    """
    Calcula un porcentaje seguro como numerator / denominator.
    - Reemplaza divisiones por cero o NaN con NaN.
    - Opcionalmente convierte a float32.
    - Puede rellenar NaNs con `fillna`.
    """
    result = (numerator / denominator).mask((denominator == 0) | (denominator.isna()))
    if fillna is not None:
        result = result.fillna(fillna)
    return result.astype(dtype)

In [27]:
def reduce_mem_usage(df):
    """Itera por las columnas del DataFrame y modifica el tipo de datos para reducir uso de memoria."""
    start_mem = df.memory_usage().sum() / 1024**2
    print(f'Uso de memoria inicial del DataFrame: {start_mem:.2f} MB')

    for col in df.columns:
        col_type = df[col].dtype

        if pd.api.types.is_numeric_dtype(col_type):
            c_min = df[col].min()
            c_max = df[col].max()

            if pd.api.types.is_integer_dtype(col_type):
                if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                else:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min >= np.finfo(np.float16).min and c_max <= np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min >= np.finfo(np.float32).min and c_max <= np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            # Sólo convertir a categoría si no lo es ya
            if not pd.api.types.is_categorical_dtype(df[col]):
                df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print(f'Uso de memoria final del DataFrame: {end_mem:.2f} MB')
    print(f'Memoria reducida en un {(100 * (start_mem - end_mem) / start_mem):.2f}%')
    return df

In [28]:
df = pd.read_parquet("01_producto_base.parquet")

In [29]:
df = reduce_mem_usage(df)

Uso de memoria inicial del DataFrame: 18.41 MB
Uso de memoria final del DataFrame: 18.20 MB
Memoria reducida en un 1.14%


C:\Users\diana\AppData\Local\Temp\ipykernel_7720\3492438140.py:31: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df[col]):
C:\Users\diana\AppData\Local\Temp\ipykernel_7720\3492438140.py:31: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df[col]):
C:\Users\diana\AppData\Local\Temp\ipykernel_7720\3492438140.py:31: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df[col]):


In [30]:
df.shape

(31522, 300)

In [31]:
df[['periodo','product_id']].head(5)

,periodo,product_id
0,201701,20001
1,201702,20001
2,201703,20001
3,201704,20001
4,201705,20001


In [32]:
# 1. Preparación de datos (sin cambios)
df['timestamp'] = pd.to_datetime(df['periodo'], format='%Y%m')
df['item_id'] = df['product_id']
periods = sorted(df['timestamp'].unique())


# 2. Loop de simulación de pronóstico a futuro
results = []
min_length = 7
# El bucle ahora puede empezar antes y llegar hasta el final
for idx in range(min_length - 1, len(periods)):
    # CAMBIO 1: El 'cutoff' es ahora el último dato conocido en cada iteración
    cutoff = periods[idx]
    
    # CAMBIO 2: El 'target_period' se calcula, no se busca. Es 2 meses después del cutoff.
    target_period = cutoff + pd.DateOffset(months=2)

    df_train = df[df['timestamp'] <= cutoff]

    # La lógica para validar IDs y rellenar faltantes no cambia
    counts = df_train.groupby('item_id').size()
    valid_ids = counts[counts >= min_length].index.tolist()
    if not valid_ids:
        continue

    df_train = df_train[df_train['item_id'].isin(valid_ids)]
    n_meses = df_train['timestamp'].nunique()
    cutoff_str = pd.to_datetime(cutoff).strftime('%Y%m')

    ts_train = TimeSeriesDataFrame.from_data_frame(
        df_train,
        id_column='item_id',
        timestamp_column='timestamp'
    ).fill_missing_values()

    # El predictor se entrena igual, para predecir 2 pasos
    predictor = TimeSeriesPredictor(
        prediction_length=2,
        target='producto_total_tn',
        freq='MS'
    )
    predictor.fit(ts_train, num_val_windows=2, time_limit=60*60)

    # La predicción genera pronósticos para (cutoff+1 mes) y (cutoff+2 meses)
    forecast = predictor.predict(ts_train)
    df_pred = forecast['mean'].reset_index().rename(columns={
        'item_id': 'product_id',
        'mean': 'tn_pred_auto'
    })

    # CAMBIO 3: Filtramos para quedarnos solo con la predicción a 2 meses
    df_pred = df_pred[df_pred['timestamp'] == target_period]

    # Agregamos la información del contexto de la predicción
    df_pred['n_meses_hist'] = n_meses
    df_pred['periodo_pred'] = cutoff_str # Mes en que se generó la predicción
    results.append(df_pred[['product_id','periodo_pred', 'timestamp', 'tn_pred_auto', 'n_meses_hist']])

C:\Users\diana\AppData\Local\Temp\ipykernel_7720\2741242683.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['timestamp'] = pd.to_datetime(df['periodo'], format='%Y%m')
C:\Users\diana\AppData\Local\Temp\ipykernel_7720\2741242683.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['item_id'] = df['product_id']
Beginning AutoGluon training... Time limit = 3600s
AutoGluon will save models to 'c:\Maestria Ciencia de Datos\Labo 3\TP\Dataset\AutogluonModels\ag-20250713_141401'
=================== System Info =================

In [33]:
df_all_preds = pd.concat(results, ignore_index=True)

In [34]:
df_all_preds.head()

,product_id,periodo_pred,timestamp,tn_pred_auto,n_meses_hist
0,20001,201707,2017-09-01,1025.904886,7
1,20002,201707,2017-09-01,705.145762,7
2,20003,201707,2017-09-01,721.740799,7
3,20004,201707,2017-09-01,522.744958,7
4,20005,201707,2017-09-01,551.690417,7


In [35]:
df.head(15)

,periodo,product_id,producto_total_tn,avg_tn,std_tn,clientes_distintos,cust_request_qty,cust_request_tn,inicio_vida_p,fin_vida_p,...,otros_avg_lag6,otros_avg_lag7,otros_avg_lag8,otros_avg_lag9,otros_avg_lag10,otros_avg_lag11,otros_avg_lag12,otros_avg_lag13,timestamp,item_id
0,201701,20001,935.0,2.158203,13.507812,433,479,937.5,201701,201912,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-01-01,20001
1,201702,20001,798.0,1.645508,11.492188,485,432,833.5,201701,201912,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-01,20001
2,201703,20001,1303.0,2.576172,18.500000,506,509,1331.0,201701,201912,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-03-01,20001
3,201704,20001,1070.0,2.089844,17.906250,512,279,1133.0,201701,201912,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-04-01,20001
4,201705,20001,1502.0,2.927734,16.906250,513,701,1551.0,201701,201912,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-05-01,20001
5,201706,20001,1520.0,2.951172,18.218750,515,570,1576.0,201701,201912,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-06-01,20001
6,201707,20001,1031.0,1.997070,18.218750,516,381,1086.0,201701,201912,...,104.1250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-01,20001
7,201708,20001,1267.0,2.433594,14.179688,521,643,1290.0,201701,201912,...,110.4375,104.1250,NaN,NaN,NaN,NaN,NaN,NaN,2017-08-01,20001
8,201709,20001,1317.0,2.507812,17.765625,525,381,1357.0,201701,201912,...,145.2500,110.4375,104.1250,NaN,NaN,NaN,NaN,NaN,2017-09-01,20001
9,201710,20001,1440.0,2.742188,23.968750,525,273,1442.0,201701,201912,...,108.6250,145.2500,110.4375,104.1250,NaN,NaN,NaN,NaN,2017-10-01,20001


In [36]:
df_all_preds.to_parquet('predicciones_autogluon.parquet', index=False)

In [37]:
df_all_preds.to_csv('predicciones_autogluon.csv', index=False)

In [38]:
df_all_preds['periodo'] = df_all_preds['timestamp'].dt.strftime('%Y%m').astype(int)


In [39]:
del df_all_preds['periodo_pred']

In [40]:
del df['timestamp']
del df['item_id']

In [41]:
df_final = df.merge(
   df_all_preds,
    on=['product_id','periodo'],
    how='left'
)

In [42]:
df_final.shape

(31522, 303)

In [43]:
df_final['tn_pred_auto_delta_a_tn']=df_final['tn_pred_auto']-df['producto_total_tn'].fillna(0)
df_final['ratio_tn_pred_auto_delta_a_tn']=percentage_safe(df_final['tn_pred_auto_delta_a_tn'],df['producto_total_tn'])
df_final['ratio_tn_pred_a_tn']=percentage_safe(df_final['tn_pred_auto'],df['producto_total_tn'])

df_final=reduce_mem_usage(df_final)

Uso de memoria inicial del DataFrame: 19.40 MB
Uso de memoria final del DataFrame: 18.53 MB
Memoria reducida en un 4.49%


C:\Users\diana\AppData\Local\Temp\ipykernel_7720\3492438140.py:31: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df[col]):
C:\Users\diana\AppData\Local\Temp\ipykernel_7720\3492438140.py:31: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df[col]):
C:\Users\diana\AppData\Local\Temp\ipykernel_7720\3492438140.py:31: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df[col]):
C:\Users\diana\AppData\Local\Temp\ipykernel_7720\3492438140.py:31: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.Categorical

In [50]:
df_final[['periodo','product_id','producto_total_tn','tn_pred_auto','tn_pred_auto_delta_a_tn','ratio_tn_pred_auto_delta_a_tn','ratio_tn_pred_a_tn','n_meses_hist','clase_producto']].head(50)

,periodo,product_id,producto_total_tn,tn_pred_auto,tn_pred_auto_delta_a_tn,ratio_tn_pred_auto_delta_a_tn,ratio_tn_pred_a_tn,n_meses_hist,clase_producto
0,201701,20001,935.0,NaN,NaN,NaN,NaN,NaN,368.0
1,201702,20001,798.0,NaN,NaN,NaN,NaN,NaN,272.0
2,201703,20001,1303.0,NaN,NaN,NaN,NaN,NaN,199.0
3,201704,20001,1070.0,NaN,NaN,NaN,NaN,NaN,450.0
4,201705,20001,1502.0,NaN,NaN,NaN,NaN,NaN,-471.0
5,201706,20001,1520.0,NaN,NaN,NaN,NaN,NaN,-253.0
6,201707,20001,1031.0,NaN,NaN,NaN,NaN,NaN,286.0
7,201708,20001,1267.0,NaN,NaN,NaN,NaN,NaN,173.0
8,201709,20001,1317.0,1026.0,-291.000000,-0.221069,0.778809,7.0,263.0
9,201710,20001,1440.0,1153.0,-287.250000,-0.199463,0.800781,8.0,-391.0


# Exportar

In [70]:
df_final.to_parquet('1_b_producto_autogluon.parquet', index=False)

In [69]:

df_final.head(1000).to_excel('01b_producto_autogluon.xlsx',sheet_name='hoja1',index=False)

In [46]:
#df_train = df[~df['periodo'].isin([201911, 201912])]
#df_train = df_final.query("periodo != 201911 and periodo != 201912")

#df_train.to_parquet('train_producto_autogluon.parquet', index=False)
print(f"DataFrame de entrenamiento guardado en 'train.parquet' con {len(df_train)} filas.")

# --- 3. Preparar y guardar el DataFrame de predicción en Parquet ---
# Seleccionamos los periodos 201911 y 201912 para el conjunto de predicción.
# Eliminamos la columna 'clase' ya que no será necesaria para la predicción.
# Finalmente, guardamos este DataFrame en un archivo Parquet.
#df_predecir = df_final[df_final['periodo'].isin([201911, 201912])].copy() # Usar .copy() para evitar SettingWithCopyWarning
#df_predecir.drop(columns=['clase'], inplace=True)
#df_predecir.to_parquet('predecir_producto_autogluon.parquet', index=False)
#print(f"DataFrame para predicción guardado en 'predecir.parquet' con {len(df_predecir)} filas.")

DataFrame de entrenamiento guardado en 'train.parquet' con 31021 filas.
